# Hand Detection and Finger Counting

In this project, with the help of openCV, we create a program which detects the hand and counts the number of fingers being held up. 

In [2]:
#Importing the required libraries
import cv2
import numpy as np

from sklearn.metrics import pairwise

In [3]:
#Creating some global variables
background=None

accumulated_weight = 0.5

roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600


In [4]:
def calc_accum_avg(frame,accumulated_weight):
    global background
    
    if background is None:
        background = frame.copy().astype('float')
        #Returns None only the first time, when background is None
        return None
    #Subsequently returns nothing
    cv2.accumulateWeighted(frame,background,accumulated_weight) 

Here, we utilize thresholding to obtain the hand segment from the region of interest (ROI).

In [5]:
def segment(frame, threshold = 25):
    
    #Compute the absolute difference between the frame and background
    diff = cv2.absdiff(background.astype('uint8'),frame)
    
    ret, thresh = cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY)
    
    image,contours,hierarchy = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours)==0: return None
    
    #The largest external countor in the ROI is expected to be the hand
    handSegment = max(contours,key=cv2.contourArea)
    
    return thresh, handSegment

**Counting Fingers via Convex Hull**

A convex hull is the smallest convex polygon that contains all the points in a frame, as shown in the figure below:

![](Capture.PNG)

We obtain the convex hull of the thresholded image of our hand:

![](download.jpg)

To count the number of fingers being held up, we begin by: 

I) Determine the most extreme top, bottom, left, and right points of the convex hull perimeter.

II) Compute their intersection and estimate this as the center of the hand.

III) Compute the distance $d$ of the point furthest away from this center.

IV) Draw a circle centered at the intersection, with radius $\gamma d$, where $0 \leq \gamma \leq 1$.

The idea is this: Any points on the convex hull which lie outside this circle and are far from the bottom of the frame (so we don't erroneously detect our wrist) are likely to be tips of extended fingers.

In [6]:
def fingerCount(thresh,hand,gamma):
    
    #Compute the convex hull of the hand
    hull = cv2.convexHull(hand)
    
    #Find the top, bottom, left , and right.
    #Then make sure they are in tuple format
    top    = tuple(hull[hull[:, :, 1].argmin()][0])
    bottom = tuple(hull[hull[:, :, 1].argmax()][0])
    left   = tuple(hull[hull[:, :, 0].argmin()][0])
    right  = tuple(hull[hull[:, :, 0].argmax()][0])

    #We compute the coordinates of the center
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2
    dist = pairwise.euclidean_distances([(cX,cY)], Y=[left,right,top,bottom])[0]
    max_dist = dist.max()
    
    radius = int(gamma*max_dist)
    circumference = 2*np.pi*radius
    
    #Defining the region of interest
    circ_roi = np.zeros(thresh.shape[:2], dtype="uint8")
    
    cv2.circle(circ_roi,(cX,cY),radius,255,10)
    
    circ_roi = cv2.bitwise_and(thresh,thresh,mask=circ_roi)
    
    #Getting the contours in our ROI
    image, contours, hierarchy = cv2.findContours(circ_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    #We count the points that lie outside the circle
    count = 0
    
    for a in contours:
        
        #Getting the bounding box of the contour
        (x,y,w,h) = cv2.boundingRect(a)
        
        not_wrist = (cY+(cY*0.25)) > (y+h)
        
        within = ((circumference*0.25)>a.shape[0])
        
        if not_wrist and within:
            count+=1
            
    return count

In [7]:
pairwise.euclidean_distances([(3,4)],[(4,5),(5,6)])

array([[1.41421356, 2.82842712]])

**Running the Program**

In [23]:
cam = cv2.VideoCapture(0)

num_frames = 0

width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

#Creating the video writer object
writer = cv2.VideoWriter('hand_capture.mp4', cv2.VideoWriter_fourcc(*'VIDX'),25, (width, height))

while True:
    
    ret, frame = cam.read()
    
    frame = cv2.flip(frame,1)
    
    frame_copy = frame.copy()
    
    roi = frame[roi_top:roi_bottom,roi_right:roi_left]
    
    #Converting our image to grayscale
    grayscale = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    
    #Blurring the image using a Gaussian kernel
    grayscale = cv2.GaussianBlur(grayscale,(7,7),0)
    
    #Getting the background from the first 60 frames
    if num_frames < 60:
        calc_accum_avg(grayscale, accumulated_weight)
        
        if num_frames <= 59:
            cv2.putText(frame_copy, 'Getting background...', (200,400),cv2.FONT_HERSHEY_DUPLEX,1,(0,255,0),2)
            cv2.imshow('Finger count',frame_copy)
    else:
        hand = segment(grayscale)
        
        if hand:
            thresh, handSegment = hand
         
            #We draw contours around the hand during the live stream
            cv2.drawContours(frame_copy,[handSegment+(roi_right,roi_top)],-1,(255,0,0),5)
        
            num_fingers = fingerCount(thresh, handSegment, 0.85)
        
            cv2.putText(frame_copy, str(num_fingers),(70,45),cv2.FONT_HERSHEY_DUPLEX,1,(0,0,255),2)
        
            cv2.imshow('Thresholded', thresh)
    
    #Writing the video to file
    writer.write(frame_copy)
    #Drawing the rectangular ROI
    cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)
    
    num_frames += 1
    
    cv2.imshow('Finger count',frame_copy)
    
    k = cv2.waitKey(1) & 0xFF
    
    if k==27:
        break
    
cam.release()
writer.release()
cv2.destroyAllWindows()